# Feature Engineering
Se busca reemplazar en el archivo 'Clean_australian_user_reviews.parquet' la columna 'review' por una columna llamada 'sentiment_analysis'. Esta nueva columna tendrá para cada celda uno de tres posible valores: 0, si el review es desfavorable; 1, si el review es neutro o no hay review; y 2, si el review es favorable.

Con lo anterior se brindará mejores datos al modelo de ML, esperando obtener a cambio mejores recomendaciones de juegos.

Este proceso de Feature Engineering comprende los siguientes pasos:
1. Detección del idioma predominante.
2. Pre-procesamiento de datos. 
3. Análisis de sentimiento.
4. Eliminación de la columna 'review' y creación del archivo definitivo 'Clean_australian_user_reviews_FE.parquet'.

## Detección del idioma predominante
Parte importante del trabajo en el segundo paso requiere que se sepa cual es el lenguaje predominante en la columna 'review' que será reemplazada por la columna 'sentiment_analysis'.

Del análisis exploratorio inicial que se hizo sobre los archivos recibidos originalmente, era evidente que el lenguaje usado en 'review' es inglés, tal como se observa en la porción del dataframe mostrado lineas abajo. Sin embargo, en su momento se pudo apreciar la presencia de otros carácteres de lenguas no occidentales.

Se usará la librería 'langdetect' para obtener una distribución de frecuencia de las lenguas usadas para escribir los  distintos 'review' encontrados en el archivo 'Clean_australian_user_reviews.parquet'.

In [1]:
import pandas as pd

# Carga el archivo Parquet 'Clean_australian_user_reviews' en DataFrame 'df1'
df1 = pd.read_parquet('../Datasets/Clean_Parquet_Data_Steam/Clean_australian_user_reviews.parquet')

# Presenta una muestra del contenido de df1
df1

,posted,item_id,recommend,review,user_id,user_url
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637
4,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637
...,...,...,...,...,...,...
58426,Posted July 10.,70,True,a must have classic from steam definitely wort...,76561198312638244,http://steamcommunity.com/profiles/76561198312...
58427,Posted July 8.,362890,True,this game is a perfect remake of the original ...,76561198312638244,http://steamcommunity.com/profiles/76561198312...
58428,Posted July 3.,273110,True,had so much fun plaing this and collecting res...,LydiaMorley,http://steamcommunity.com/id/LydiaMorley
58429,Posted July 20.,730,True,:D,LydiaMorley,http://steamcommunity.com/id/LydiaMorley


In [2]:
from langdetect import detect

# Función para detectar el idioma predominante en el texto
def detect_language(text):
    if pd.isnull(text):  # Si no hay texto de revisión
        return None  # Devolver None si no hay texto
    else:
        try:
            return detect(str(text))
        except:
            return None

# Aplicar la función a la columna 'review' para detectar el idioma
df1['language'] = df1['review'].apply(detect_language)

# Cantidad de registros por idioma detectado
language_counts = df1['language'].value_counts()

# Cantidad de registros por idioma detectado en porcentaje
language_distribution = df1['language'].value_counts(normalize=True) * 100

# Crea un DataFrame con language_counts y language_distribution
df_lang_dist = pd.DataFrame({'Counts': language_counts, 'Dist (%)': language_distribution})
df_lang_dist

,Counts,Dist (%)
language,,
en,45099,77.947734
pt,2152,3.719451
es,1259,2.176017
de,1127,1.947872
so,1017,1.757752
af,733,1.266895
th,708,1.223686
tl,574,0.992084
cy,462,0.798507


Con 77,9% del total, 'en' (English) se lleva de lejos el primer lugar al lenguaje más usado para suscribir los reviews en la plataforma de juegos de STEAM.


## Pre-procesamiento de datos
El propósito del pre-procesamiento es eliminar de los datos en la columna 'review' tanto ruido como se pueda. En este sentido, se ponen todos los textos en minúscula, se eliminan signos de puntuación, se eliminan stop-words o palabras vacías que no agregan valor semántico o de sentimiento al mensaje, y se lematizan las palabras para llevarlas a su forma semántica base.

## Análisis de sentimiento
Usaremos el 'polarity_scores' proporcionado por el módulo SentimentIntensityAnalyzer de NLTK (Natural Language Toolkit) para realizar análisis de sentimientos en texto. Este 'polarity_scores' calcula las puntuaciones de polaridad del texto, es decir, evalúa la positividad, negatividad, neutralidad y la compuesta (compound) del texto analizado.

La puntuación compuesta (compound) se utilizará para determinar la polaridad general del texto, ya que representa una combinación de las demás en una sola métrica. Estas puntuaciones oscilan entre -1 y 1, donde:

* Valores cercanos a 1 indican una polaridad positiva.
* Valores cercanos a -1 indican una polaridad negativa.
* Valores cercanos a 0 indican neutralidad.

Tanto el pre-procesamiento como el Análisis de sentimiento lo realizaremos usando la función 'GetSentiment()' presentada a continuación.


In [3]:
# Importa la librería NLTK para procesamiento de lenguaje natural
import nltk
# Importa el analizador de sentimientos de NLTK
from nltk.sentiment import SentimentIntensityAnalyzer
# Importa la función de tokenización de NLTK
from nltk.tokenize import word_tokenize
# Importa el conjunto de stopwords de NLTK
from nltk.corpus import stopwords
# Importa el lematizador de NLTK
from nltk.stem import WordNetLemmatizer
# Importa el módulo de string de Python que usaremos para manejo de caracteres de puntuación
import string
# Importa la librería Pandas para manipulación de datos en tablas (DataFrames)
import pandas as pd

# Descarga los recursos necesarios para tokenización de texto
nltk.download('punkt')
# Descarga el conjunto de stopwords
nltk.download('stopwords')
# Descarga WordNet, un diccionario léxico de inglés
nltk.download('wordnet')
# Descarga el léxico VADER para análisis de sentimientos
nltk.download('vader_lexicon')

# Inicializa el analizador de sentimientos
sid = SentimentIntensityAnalyzer()

# Inicializa los stopwords
stop_words = set(stopwords.words('english'))

# Inicializa el lematizer
lemmatizer = WordNetLemmatizer()

# Recibe el texto de un review y devuelve 0 si lo interpreta como desfavorable; 1, para neutral o en caso de ausencia de 
# texto en el campo 'review'; y 2, para favorable. 
# Antes de analizar el texto lo pre-procesa para mejorar la consistencia de los análisis.
def GetSentiment(text):
    # Si el texto del review es nulo
    if pd.isnull(text):  
        return 1 
    else:
        # Convierte a minúsculas y tokeniza el texto
        words = word_tokenize(text.lower())  

        # Elimina signos de puntuación
        words = [word for word in words if word not in string.punctuation]

        # Elimina palabras sin valor semántico (vacías o stopwords)
        words = [word for word in words if word not in stop_words]

        # Lematización para llevar las palabras a su forma base (saltó --> saltar, saltando --> saltar, etc.)
        words = [lemmatizer.lemmatize(word) for word in words]

        # Une las palabras procesadas en un solo texto
        processed_text = ' '.join(words)

        # Realiza análisis de sentimientos usando NLTK
        ss = sid.polarity_scores(processed_text)
        compound_score = ss['compound']

        # Asigna el valor según el análisis de sentimientos
        if compound_score >= 0.05: 
            return 2
        elif compound_score <= -0.05: 
            return 0
        else:  
            return 1

# Construye la columna 'sentiment_analysis' aplicando la función GetSentiment() a los elementos de la columna 'review' 
df1['sentiment_analysis'] = df1['review'].apply(GetSentiment)
df1

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\NAZG\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\NAZG\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\NAZG\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\NAZG\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,posted,item_id,recommend,review,user_id,user_url,language,sentiment_analysis
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...,en,2
1,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...,en,2
2,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...,en,2
3,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637,en,2
4,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637,en,2
...,...,...,...,...,...,...,...,...
58426,Posted July 10.,70,True,a must have classic from steam definitely wort...,76561198312638244,http://steamcommunity.com/profiles/76561198312...,en,2
58427,Posted July 8.,362890,True,this game is a perfect remake of the original ...,76561198312638244,http://steamcommunity.com/profiles/76561198312...,en,2
58428,Posted July 3.,273110,True,had so much fun plaing this and collecting res...,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,en,2
58429,Posted July 20.,730,True,:D,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,de,1


## Eliminación de la columna 'review' y creación del archivo definitivo 'Clean_australian_user_reviews_FE.parquet'

In [4]:
# Elimina la columna 'review' del DataFrame df1
df1.drop('review', axis=1, inplace=True)

# Guarda los datos de df1 en archivo Clean_australian_user_reviews_FE.parquet
df1.to_parquet('../Datasets/Clean_Parquet_Data_Steam/Clean_australian_user_reviews_FE.parquet')
